## MBA em IA e BIGDATA
## SCC-ICMC-USP
## **CURSO 2 - CD_AM_DM**
## **Exemplo - IMPUTAÇÃO**

### Profa. Roseli A. F. Romero


**OBJETIVO**:
## Mostrar a função do sklearn que realiza o preenchimento de
##dados faltantes


# Objetivo

A ideia deste notebook é demonstrar como podemos fazer transformações em dados heterogêneos com o sklearn usando SimpleImputer, Pipeline e ColumnTransformer. A ideia é facilitar a manipulação de tais tipos de dados, que são comuns no dia-a-dia.



In [6]:
import pandas as pd
import numpy as np
# Dados de exemplo
dados = pd.DataFrame({
    'idade': [15, 23, 19, 30, 44, np.nan],
    'sexo': ['homem', 'mulher', 'homem', 'homem', np.nan,'homem'],
    'altura': [1.6, 1.7, np.nan, 1.8, 1.75, 1.65],
    'classe':  [-1, 1, -1, 1, 1, -1]
    })
dados

,idade,sexo,altura,classe
0,15.0,homem,1.60,-1
1,23.0,mulher,1.70,1
2,19.0,homem,NaN,-1
3,30.0,homem,1.80,1
4,44.0,NaN,1.75,1
5,NaN,homem,1.65,-1


In [7]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   idade   5 non-null      float64
 1   sexo    5 non-null      object 
 2   altura  5 non-null      float64
 3   classe  6 non-null      int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 320.0+ bytes


A partir do comando `info()` nota-se que existem 2 variáveis de entrada numéricas (idade e altura) e 1 variável de entrada categórica (sexo). A variável classe é a variável alvo.

### Transformando os dados

Considere que queremos realizar as seguintes operações:


*   **Nos dados numéricos:** substituir os dados faltantes das variáveis pela média dos valores presentes. Depois padronizar o intervalo dessas variáveis
*   **Nos dados categóricos:** Substituir os valores faltantes pelo valor mais frequente. Depois transformar essas categorias em um atributo numérico para usar no nosso modelo de rede neural

Para fazer essas operações em colunas específicas, podemos utilizar as ferramentas `sklearn.pipeline.Pipeline` e `sklearn.compose.ColumnTransformer`.

A ferramenta `sklearn.pipeline.Pipeline` cria uma sequencia de transformações nos dados, enquanto que a `sklearn.compose.ColumnTransformer` realiza uma transformação em dados de uma coluna.

Para substituir valores faltantes utilizaremos a classe `sklearn.impute.SimpleImputer`, para padronizar os dados `sklearn.preprocessingStandardScaler` e para converter atributos categoricos em valores numéricos `sklearn.preprocessing.OneHotEncoder`.

Aplicando nos dados do exemplo acima:



In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Criamos um vetor com o nome das classes desejadas
features_numericos = ['idade', 'altura']
features_categoricos = ['sexo']

# Criando os pipelines
pipeline_numerico = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))])
#     ('scaler', StandardScaler())])

pipeline_categorico = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])
#    ('onehot', OneHotEncoder())])

# Criando a transformação do conjunto de dados:
transformacao = ColumnTransformer(
    transformers=[
        ('transformacao numerica', pipeline_numerico, features_numericos),
        ('transformacao categorica', pipeline_categorico, features_categoricos),
    ])

# Aplicando a transformação no dataset:
dados_transformados = transformacao.fit_transform(dados)
#dados_transformados.round(2)

In [9]:
dados_transformados

array([[15.0, 1.6, 'homem'],
       [23.0, 1.7, 'mulher'],
       [19.0, 1.7, 'homem'],
       [30.0, 1.8, 'homem'],
       [44.0, 1.75, 'homem'],
       [26.2, 1.65, 'homem']], dtype=object)

Note que a ordem das variáveis (features) mudou em relação ao nosso conjunto inicial. A ordem do conjunto transformado é dada pela ordem de processamento das features. No nosso caso, a ordem das features ficou:

(features_numericos, features_categoricos) $\rightarrow$ (idade, altura, sexo)

 Note também que a transformação `OneHotEncoder` vai transformar os features categóricos em um vetor unitário, que possui $n$ dimensões, onde $n$ é o número de valores diferentes da categoria (no nosso exemplo, 2 - homem e mulher)

 Observe que o conjunto não possui mais a variável "classe". Isso porque não foi efetuada nenhuma transformação nessa variável.

 Normalmente para problemas de Machine Learning é comum separar as variáveis de entrada ("sexo", "idade" e "altura") da variável alvo ("classe"). Mas se for necessário, podemos facilmente concatenar a variável classe e criar uma estrutura `DataFrame`:

In [10]:
dados_transformados_com_classe = np.c_[dados_transformados, dados['classe']]
dataframe_processado = pd.DataFrame(dados_transformados_com_classe)
dataframe_processado

,0,1,2,3
0,15.0,1.6,homem,-1
1,23.0,1.7,mulher,1
2,19.0,1.7,homem,-1
3,30.0,1.8,homem,1
4,44.0,1.75,homem,1
5,26.2,1.65,homem,-1
